In [23]:
import numpy as np
import torch
from torch.utils.data import DataLoader

xs = np.random.randint(1, 100, (1, 2, 3))
print(xs)

print(xs.shape)

xs = torch.tensor(xs, dtype=torch.float32)

print(xs.shape)



dataloader = DataLoader(xs, batch_size=1)

for x in (dataloader):
  print(x)





[[[86 71 29]
  [46 60 88]]]
(1, 2, 3)
torch.Size([1, 2, 3])
tensor([[[86., 71., 29.],
         [46., 60., 88.]]])


In [24]:
from data.datasets import MedicalDecathlonDataset
from omegaconf import OmegaConf
import os
import torch

cfg = OmegaConf.load("conf/config.yaml")
task_name = cfg.dataset.task_name
dataset_path = f"{cfg.dataset.base_path}{task_name}/"
images_path = f"{dataset_path}{cfg.dataset.images_subdir}"
labels_path = f"{dataset_path}{cfg.dataset.labels_subdir}"
target_shape = cfg.dataset.target_shape




dataset = MedicalDecathlonDataset(cfg, task_name, images_path, labels_path, target_shape)

labels = set()
for x, y in dataset:
  print(y.shape)
  break
print(labels)


all_labels = set()
for _, y in dataset:
    # Get unique values from the tensor and convert to a Python list
    unique_vals = torch.unique(y).tolist()
    for val in unique_vals:
        all_labels.add(val)

print("Unique labels:", all_labels)
print("Number of unique labels:", len(all_labels))

all_labels = set()
for _, y in dataset:
    for pixel in y.flatten().tolist():
        all_labels.add(pixel)

print("Unique labels:", all_labels)
print("Number of unique labels:", len(all_labels))


ConfigAttributeError: Missing key dataset
    full_key: dataset
    object_type=dict

In [ ]:
from data.datasets import MedicalDecathlonDataset
from omegaconf import OmegaConf
import os
import nibabel as nib

cfg = OmegaConf.load("conf/config.yaml")
task_name = cfg.dataset.task_name
dataset_path = f"{cfg.dataset.base_path}{task_name}/"
images_path = f"{dataset_path}{cfg.dataset.images_subdir}"
labels_path = f"{dataset_path}{cfg.dataset.labels_subdir}"
target_shape = cfg.dataset.target_shape


label_path = os.path.join(labels_path, sorted(os.listdir(labels_path))[0])

label = nib.load(label_path).get_fdata() # Shape: [W, H, D]
print(label.min(), label.max())
print(label.shape)




In [29]:
import numpy as np
from omegaconf import DictConfig, OmegaConf
from typing import Dict, Tuple

cfg = OmegaConf.load("conf/config.yaml")
task_name = cfg.active_dataset


def merge_dataset_config(cfg: DictConfig) -> Tuple[DictConfig, DictConfig]:
    """
    Merges default training settings with dataset-specific overrides.
    """
    dataset_cfg: DictConfig = cfg.datasets[cfg.active_dataset]
    overrides = dataset_cfg.get("training_overrides", {})
    merged_training = OmegaConf.merge(cfg.training_defaults, overrides)
    
    if not isinstance(merged_training, DictConfig):
        raise TypeError("Merged training config is not a DictConfig. Please check your configuration structure.")
    
    return dataset_cfg, merged_training

print(merge_dataset_config(cfg))


({'target_shape': [36, 52, 36], 'base_path': 'datasets/Task04_Hippocampus/', 'images_subdir': 'imagesTr', 'labels_subdir': 'labelsTr', 'num_classes': 2}, {'batch_size': 2, 'learning_rate': 0.001, 'weight_decay': 1e-07, 'num_epochs': 100, 'n_filters': 16, 'dropout': 0.05, 'drop_last': True})
